In [1]:
import gym
import torch
import random
import warnings
import itertools
import numpy as np
import pandas as pd
import torch.nn as nn
from typing import Sequence
import matplotlib.pyplot as plt
import torch.nn.functional as F
from collections import namedtuple, deque
from plot_script import plot_result, draw_neural_net
from torch.distributions.categorical import Categorical

warnings.filterwarnings("ignore")

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [2]:
GAMMA = 0.99
BATCH_SIZE = 64
BUFFER_SIZE = 10000
MIN_REPLAY_SIZE = 5000
EPS_START = 1.0
EPS_END = 0.05
EPS_DECAY = 0.995
TARGET_UPDATE_FREQ = 5

In [3]:
env = gym.make("LunarLander-v2")
obs = env.reset()
episode_reward = 0.0

DependencyNotInstalled: box2D is not installed, run `pip install gym[box2d]`

In [8]:
pip install box2d-py

     ---------------------------------------- 0.0/374.5 kB ? eta -:--:--
     - -------------------------------------- 10.2/374.5 kB ? eta -:--:--
     --- --------------------------------- 30.7/374.5 kB 330.3 kB/s eta 0:00:02
     ------- ----------------------------- 71.7/374.5 kB 491.5 kB/s eta 0:00:01
     -------------------- ----------------- 204.8/374.5 kB 1.1 MB/s eta 0:00:01
     ---------------------------------- --- 337.9/374.5 kB 1.4 MB/s eta 0:00:01
     -------------------------------------- 374.5/374.5 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for box2d-py
Failed to build box2d-py
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [16 lines of output]
  Using setuptools (version 68.0.0).
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-cpython-311
  creating build\lib.win-amd64-cpython-311\Box2D
  copying library\Box2D\Box2D.py -> build\lib.win-amd64-cpython-311\Box2D
  copying library\Box2D\__init__.py -> build\lib.win-amd64-cpython-311\Box2D
  creating build\lib.win-amd64-cpython-311\Box2D\b2
  copying library\Box2D\b2\__init__.py -> build\lib.win-amd64-cpython-311\Box2D\b2
  running build_ext
  building 'Box2D._Box2D' extension
  swigging Box2D\Box2D.i to Box2D\Box2D_wrap.cpp
  swig.exe -python -c++ -IBox2D -small -O -includeall -ignoremissing -w201 -globals b2Globals -outdir library\Box2D -keyword -w511 -D_SWIG_KWARGS -o Box2D\Box2D_wrap.cpp Box2D\Box2D.i
  error: command 'swig.exe' failed: None
  [end of output]
  
  note: This error ori

In [7]:
pip install pygame==2.1.0 --no-build-isolation

  Using cached pygame-2.1.0.tar.gz (5.8 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [77 lines of output]
  
  
  WARNING, No "Setup" File Exists, Running "buildconfig/config.py"
  Using WINDOWS configuration...
  
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\Nicholas Vella\AppData\Local\Temp\pip-install-g_bhxd8d\pygame_5cf1490794324a61a2cc6ed502918db3\setup.py", line 388, in <module>
      buildconfig.config.main(AUTO_CONFIG)
    File "C:\Users\Nicholas Vella\AppData\Local\Temp\pip-install-g_bhxd8d\pygame_5cf1490794324a61a2cc6ed502918db3\buildconfig\config.py", line 234, in main
      deps = CFG.main(**kwds)
             ^^^^^^^^^^^^^^^^
    File "C:\Users\Nicholas Vella\AppData\Local\Temp\pip-install-g_bhxd8d\pygame_5cf1490794324a61a2cc6ed502918db3\buildconfig\config_win.py", line 511, in main
      return setup_prebuilt_sdl2(prebuilt_dir)
        

In [ ]:
Transition = namedtuple('Transition', ('states', 'actions', 'rewards', 'dones', 'next_states'))

class Replay_memory():

    def __init__(self, env, fullsize, minsize, batchsize):
        self.env = env
        self.memory = deque(maxlen=fullsize)
        self.rewards = deque(maxlen=50)
        self.batchsize = batchsize
        self.minsize = minsize

    def append(self, transition):
        self.memory.append(transition)
    
    def sample_batch(self):
        batch = random.sample(self.memory, self.batchsize)
        batch = Transition(*zip(*batch))
        states = torch.from_numpy(np.array(batch.states, dtype=np.float32)).to(device)
        actions = torch.from_numpy(np.array(batch.actions, dtype=np.int64)).unsqueeze(1).to(device)
        rewards = torch.from_numpy(np.array(batch.rewards, dtype=np.float32)).unsqueeze(1).to(device)
        dones = torch.from_numpy(np.array(batch.dones, dtype=np.bool8)).unsqueeze(1).to(device)
        next_states = torch.from_numpy(np.array(batch.next_states, dtype=np.float32)).to(device)
        
        return states, actions, rewards, dones, next_states

    def initialize(self):
        obs = env.reset()
        for _ in range(self.minsize):
            action = self.env.action_space.sample()
            new_obs, reward, done, info = env.step(action)
            transition = Transition(obs, action, reward, done, new_obs)
            self.append(transition)
            obs = new_obs
            if done:
                self.env.reset()
        return self

In [ ]:
replay_memory = Replay_memory(env, BUFFER_SIZE, MIN_REPLAY_SIZE, BATCH_SIZE).initialize()

In [ ]:
class DQN(nn.Module):
    def __init__(self, ninputs, noutputs):
        super(DQN, self).__init__()
        self.a1 = nn.Linear(ninputs, 64, device=device)
        self.a2 = nn.Linear(64, noutputs, device=device)
    
    def forward(self, X):
        o = self.a1(X)
        o = torch.tanh(o)
        o = self.a2(o)
        
        return o
    
    def __call__(self, X):
        return self.forward(X)

In [ ]:
dqn_policy = DQN(env.observation_space.shape[0], env.action_space.n)
dqn_target = DQN(env.observation_space.shape[0], env.action_space.n)
dqn_target.load_state_dict(dqn_policy.state_dict())
dqn_target.eval()

In [ ]:
loss_fn = nn.SmoothL1Loss()
learning_rate = 0.001
optimizer = torch.optim.Adam(dqn_policy.parameters(), lr=learning_rate)

In [ ]:
def epsilon_greedy_policy(epsilon, obs):
    rnd_sample = random.random()
    if rnd_sample <= epsilon:
        action = env.action_space.sample()
    else:
        with torch.no_grad():
            obs = torch.Tensor(obs).to(device)
            action = int(torch.argmax(dqn_policy(obs)))
    return action

In [ ]:
obs = env.reset()
eps_threshold = EPS_START
episode = 1
history = []

for step in itertools.count():
    
    #Get action using Epsilon-Greedy Policy
    action = epsilon_greedy_policy(eps_threshold, obs)
    
    #Get the new observation and reward.
    new_obs, reward, done, _ = env.step(action)
    
    #Append to Replay Memory
    replay_memory.append(Transition(obs, action, reward, done, new_obs))
    episode_reward += reward
    obs = new_obs
    
    #If the episode is finished
    if done:
        
        episode += 1
        eps_threshold = np.max((eps_threshold*EPS_DECAY, EPS_END))
        replay_memory.rewards.append(episode_reward)
        obs = env.reset()
        avg_res = np.mean(replay_memory.rewards)
        history.append((episode-1, avg_res))
        
                
        if episode % 100 == 0: 
            print(f'Episode: {episode} Avg Results: {avg_res} Epsilon: {eps_threshold}')

        if avg_res >= 195:
            print(f'Solved at episode: {episode} Avg Results: {avg_res}')
            break
        
        if step % TARGET_UPDATE_FREQ == 0:
            dqn_target.load_state_dict(dqn_policy.state_dict())

        episode_reward = 0
    
    #Sample from the Replay Memory
    b_states, b_actions, b_rewards, b_dones, b_next_states = replay_memory.sample_batch()
    
    #Get Q-Values of every state-action pair from the Replay Memory Sample
    qvalues = dqn_policy(b_states).gather(1, b_actions)
    
    #Train the Neural Network to better evaluate the states observed.
    with torch.no_grad():
        target_qvalues = dqn_target(b_next_states)
        max_target_qvalues = torch.max(target_qvalues, axis=1).values.unsqueeze(1)
        expected_qvalues = b_rewards + GAMMA * (1 - b_dones.type(torch.int64)) * max_target_qvalues

    loss = loss_fn(qvalues, expected_qvalues)
    optimizer.zero_grad()
    loss.backward()
    for param in dqn_policy.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()
    

In [ ]:
history = pd.DataFrame(
    history, columns=['Episode', 'Score'])

(fig, ax) = plt.subplots(1, 1)

ax.plot(history['Episode'], history['Score'])

ax.set_xlabel('Epoch')
ax.set_ylabel('Score over last 50 episodes')

plt.show()